In [90]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import re
import pandas as pd
from multiprocessing import Pool
from itertools import chain
import numpy as np
import requests

In [91]:
url = "https://www.open-medicaments.fr/api/v1/medicaments?query=ibuprof%C3%A8ne"
response_med= requests.get(url=url).json()
names = []
ids = []
for i in range(0, len(response_med)):
    ids.append(response_med[i]["codeCIS"])
    names.append(response_med[i]["denomination"])
names

['IBUPROFENE ARROW 5 %, gel',
 'IBUPROFENE URGO 5 %, gel',
 'IBUPROFENE MYLAN 5 % Solution pour pulvérisation cutanée',
 'IBUPROFENE SANDOZ 200 mg, comprimé enrobé',
 'IBUPROFENE ARROW 200 mg, comprimé enrobé',
 'IBUPROFENE TEVA 200 mg, comprimé enrobé',
 'IBUPROFENE ZYDUS 200 mg, comprimé enrobé',
 'IBUPROFENE TEVA CONSEIL 200 mg, comprimé enrobé',
 'IBUPROFENE ALMUS 200 mg, comprimé pelliculé',
 'IBUPROFENE MYLAN 200 mg, comprimé enrobé',
 'IBUPROFENE RATIOPHARM 200 mg, comprimé enrobé',
 'IBUPROFENE BIOGARAN CONSEIL 200 mg, comprimé pelliculé',
 'IBUPROFENE CRISTERS 200 mg, comprimé pelliculé',
 'IBUPROFENE BIOGARAN 200 mg, comprimé pelliculé',
 'IBUPROFENE CRISTERS 200 mg, comprimé enrobé',
 'IBUPROFENE EG 200 mg, comprimé pelliculé',
 'IBUPROFENE SANDOZ CONSEIL 200 mg, comprimé enrobé',
 'IBUPROFENE RPG 400 mg, comprimé pelliculé',
 'IBUPROFENE EVOLUGEN 400 mg, comprimé pelliculé',
 'IBUPROFENE ARROW 400 mg, comprimé pelliculé']

In [92]:
url2 = "https://www.open-medicaments.fr/api/v1/medicaments/"

In [93]:
def find_infos(ID):
    path = url2 + ID
    response_id = requests.get(url=path).json()
    amm = response_id['dateAMM']
    nom = response_id['denomination']
    age_min = re.findall(r'(\d*)\sans', str(response_id))
    if len(age_min) > 0:
        age_min = age_min[0]
    else:
        age_min = np.NAN
    poids = re.findall(r'(\d*)\smg', str(response_id['denomination']))
    if len(poids) > 0:
        poids = poids[0]
    else:
        poids = np.NAN
    return amm, nom, age_min, poids

In [94]:
df = []
for ID in ids:
    df.append(find_infos(ID))
df = pd.DataFrame(df, columns=["AMM", "Nom", "Age_Min", "poids mg"])
df

,AMM,Nom,Age_Min,poids mg
0,2006-09-19,"IBUPROFENE ARROW 5 %, gel",15,NaN
1,1995-10-13,"IBUPROFENE URGO 5 %, gel",15,NaN
2,2000-01-13,IBUPROFENE MYLAN 5 % Solution pour pulvérisati...,15,NaN
3,2001-06-06,"IBUPROFENE SANDOZ 200 mg, comprimé enrobé",6,200
4,2001-07-03,"IBUPROFENE ARROW 200 mg, comprimé enrobé",6,200
5,2002-10-30,"IBUPROFENE TEVA 200 mg, comprimé enrobé",6,200
6,2000-09-22,"IBUPROFENE ZYDUS 200 mg, comprimé enrobé",6,200
7,2002-11-19,"IBUPROFENE TEVA CONSEIL 200 mg, comprimé enrobé",6,200
8,2006-08-16,"IBUPROFENE ALMUS 200 mg, comprimé pelliculé",6,200
9,2001-12-27,"IBUPROFENE MYLAN 200 mg, comprimé enrobé",6,200


In [115]:
regex = re.compile("libelle': '(.*)'")
re.findall(r"[libelle':\s'](.*)'", str(response_id['presentations']))


["codeCIP7': '3762265', 'libelle': '1 tube(s) aluminium de 60 g', 'statutAdministratif': 'ACTIVE', 'etatCommercialisationAMM': 'Déclaration de commercialisation', 'dateDeclarationCommercialisation': '2007-11-26', 'codeCIP13': '3400937622653', 'agrementCollectivites': False, 'tauxRemboursement': [], 'prix': None, 'indicationsRemboursement': '"]